# Building an Expert System to Guide Diagnosis and Primary Management of Rhinosinusitis
## by MBMI Student Antonia Angeli Gazola
## Final project for BMIN 5200 Foundations of Artificial Intelligence in Health - Prof. Romano

In [1722]:
import clips
import sys
sys.path.append('./src/')
from clips_util import print_facts, print_rules, print_templates, build_read_assert

In [1723]:
env = clips.Environment() ## creating a CLIPS environment to be able to use CLIPS.py

import logging ## to ensure the strings are part of the output
logging.basicConfig(level=10, format='%(message)s')
router = clips.LoggingRouter()
env.add_router(router) 

## Initial Decision Track Fact Templates:

This section is to define the templates for the initial part of the decision track, that will define if the decision track is adequate for this case later on. The templates will include patient information (age, sex), pregnancy status (if female), whether predominant symptoms are upper airway and if the clinicians is suspicious of rhinosinusitis.

In [1725]:
## patient information age and sex:

DEFTEMPLATE_PATIENT = """
(deftemplate patient 
    (slot age_is (type INTEGER)
          (range 0 120))
    (slot sex_is (type SYMBOL)
          (allowed-symbols female male)))
"""
env.build(DEFTEMPLATE_PATIENT)

## If female, patient pregnancy status (will not pre-define a child bearing age limit due to variability)

DEFTEMPLATE_PREGNANCY = """
(deftemplate pregnancy_status   
    (slot preg_is (type SYMBOL)
        (allowed-symbols yes no)))
"""
env.build(DEFTEMPLATE_PREGNANCY)

## upper airway symptoms predominance

DEFTEMPLATE_UPPER_AIRWAY_SYMPTOM_PREDOMINANCE = """
(deftemplate upper_airway_predom
   (slot with_upper_airway_symp_predom (type SYMBOL)
        (allowed-symbols yes no)))
"""
env.build(DEFTEMPLATE_UPPER_AIRWAY_SYMPTOM_PREDOMINANCE)

## rhinosinusitis suspicion

DEFTEMPLATE_RHINOSINUSITIS_SUSPICION = """
(deftemplate rhinosinusitis_suspicion
   (slot with_rhinosinusitis_suspicion (type SYMBOL)
        (allowed-symbols yes no)))
"""
env.build(DEFTEMPLATE_RHINOSINUSITIS_SUSPICION)


## Rhinosinusitis Criteria Fact Templates:

This section is to define the templates for rhinosinusitis criteria questions, that will define if the case matches the criteria later on. The templates will include symptoms (general, major, minor).

In [1727]:

## checking symptoms 

DEFTEMPLATE_SYMPTOMS = """
(deftemplate symptoms 
    (slot purulent_nasal_discharge (type SYMBOL)
          (allowed-symbols yes no))
)        
"""
env.build(DEFTEMPLATE_SYMPTOMS)

## major symptoms 

DEFTEMPLATE_MAJOR_SYMPTOMS = """
(deftemplate major_symptoms 
    (slot anterior_nasal_discharge (type SYMBOL)
          (allowed-symbols yes no))
    (slot posterior_nasal_discharge (type SYMBOL)
          (allowed-symbols yes no))
    (slot nasal_congestion_obstruction (type SYMBOL)
          (allowed-symbols yes no))
    (slot facial_congestion_fullness (type SYMBOL)
          (allowed-symbols yes no))
    (slot facial_pain_pressure (type SYMBOL)
          (allowed-symbols yes no))
    (slot hyposmia_anosmia (type SYMBOL)
          (allowed-symbols yes no))
    (slot fever (type SYMBOL)
          (allowed-symbols yes no))
)        
"""
env.build(DEFTEMPLATE_MAJOR_SYMPTOMS)

## minor symptoms 

DEFTEMPLATE_MINOR_SYMPTOMS = """
(deftemplate minor_symptoms 
    (slot headache (type SYMBOL)
          (allowed-symbols yes no))
    (slot ear_pain_pressure_fullness (type SYMBOL)
          (allowed-symbols yes no))
    (slot halitosis (type SYMBOL)
          (allowed-symbols yes no))
    (slot dental_pain (type SYMBOL)
          (allowed-symbols yes no))
    (slot cough (type SYMBOL)
          (allowed-symbols yes no))
    (slot fatigue (type SYMBOL)
          (allowed-symbols yes no))
)        
"""
env.build(DEFTEMPLATE_MINOR_SYMPTOMS)


## Symptoms Duration Fact Template:

In [1729]:
## patient rhinosinusitis duration (checking if it is acute)

DEFTEMPLATE_RHINOSINUSITIS_DURATION = """
(deftemplate rhinosinusitis_duration
    (slot four_weeks_less (type SYMBOL)
    (allowed-symbols yes no))
)
"""
env.build(DEFTEMPLATE_RHINOSINUSITIS_DURATION)

## patient non-acute rhinosinusitis duration (chronic vs subacute)

DEFTEMPLATE_CHRONIC_RHINOSINUSITIS_DURATION = """
(deftemplate chronic_rhinosinusitis_duration
    (slot twelve_weeks_more (type SYMBOL)
    (allowed-symbols yes no unknown))
)
"""
env.build(DEFTEMPLATE_CHRONIC_RHINOSINUSITIS_DURATION)

## Bacterial Suspicion Template:

This section is to define the template for the bacterial suspicion check, that will check if this case has characteristics that raise suspicion for bacterial infection. The templates will include bacterial suspicion check (total presentation days, whether there is improvement, progressive worsening, double sickening, presence of 3 or more days of severe symptoms or clinical suspicion for bacterial infection based on the assessment.

In [1731]:
## bacterial suspicion checking

DEFTEMPLATE_BACTERIAL_SUSPICION_CHECK = """
(deftemplate bacterial_suspicion_check
    (slot total_presentation_days (type INTEGER)
          (range 0 30))
    (slot improvement (type SYMBOL)
          (allowed-symbols yes no))
    (slot progressive_worsening (type SYMBOL)
          (allowed-symbols yes no))
    (slot double_sickening (type SYMBOL)
          (allowed-symbols yes no))
    (slot severe_3_days (type SYMBOL)
          (allowed-symbols yes no))
    (slot bacterial_infection (type SYMBOL)
          (allowed-symbols yes no))
)
"""
env.build(DEFTEMPLATE_BACTERIAL_SUSPICION_CHECK)

## Clinical Assessment of whether Subacute Rhinosinusitis need Acute Management Template:

This section is to define the template for the question of whether a case of subacute rhinosinusitis requires acute management based on the clinical assessment, which will be checked later on.

In [1733]:
## check if clinicians considers that this subacute case requires acute management, based on clinical evaluation

DEFTEMPLATE_SUBACUTE_TTM_NEED = """
(deftemplate subacute_ttm_need
    (slot acute_ttm (type SYMBOL)
    (allowed-symbols yes no unknown))
)
"""
env.build(DEFTEMPLATE_SUBACUTE_TTM_NEED)


## Bacterial Complication Assessment Template

This section is to define the template for the question of whether the case has any bacterial complications, which will be checked later on.

In [1735]:
## bacterial complication checking

DEFTEMPLATE_BACTERIAL_COMPLICATION_CHECK = """
(deftemplate bacterial_complication_check
    (slot bact_comp (type SYMBOL)
          (allowed-symbols yes no)))
"""
env.build(DEFTEMPLATE_BACTERIAL_COMPLICATION_CHECK)

## Templates: questions to define best treatment:

This section is to define the template for the question to define the best treatment recommendation:

- whether the case is eligible for watchful waiting
- whether the case is getting worse or not improving in 7 days
- whether there is severe infection in need of hospitalization
- whether there is beta-lactam allergy
- whether the patient is clinically better
  

In [1737]:
## bacterial watchful waiting eligibility assessment

DEFTEMPLATE_WATCHFUL_WAITING_CHECK = """
(deftemplate watchful_waiting_check
    (slot watch_wait (type SYMBOL)
          (allowed-symbols yes no)))
"""
env.build(DEFTEMPLATE_WATCHFUL_WAITING_CHECK)

## bacterial worsening for 7 days assessment

DEFTEMPLATE_WORSENING_NO_IMPROVEMENT_CHECK = """
(deftemplate worsening_no_improvement_check
    (slot worse_noimprove (type SYMBOL)
          (allowed-symbols yes no)))
"""
env.build(DEFTEMPLATE_WORSENING_NO_IMPROVEMENT_CHECK)

## Checking if Severe Infection in need for Hospitalization:

DEFTEMPLATE_SEVERE_INFECTION_HOSPITAL_CHECK = """
(deftemplate severe_infection_hospital_check
    (slot severe_hospital (type SYMBOL)
          (allowed-symbols yes no)))
"""
env.build(DEFTEMPLATE_SEVERE_INFECTION_HOSPITAL_CHECK)

## Checking if Beta-lactam allergy

DEFTEMPLATE_BETA_LACTAM_ALLERGY_CHECK= """
(deftemplate beta_lactam_allergy_check
    (slot allergy (type SYMBOL)
          (allowed-symbols yes no)))
"""
env.build(DEFTEMPLATE_BETA_LACTAM_ALLERGY_CHECK)

## Checking if patient is clinically better

DEFTEMPLATE_CLINICAL_IMPROVEMENT_CHECK= """
(deftemplate clinical_improvement_check
    (slot better (type SYMBOL)
          (allowed-symbols yes no)))
"""
env.build(DEFTEMPLATE_CLINICAL_IMPROVEMENT_CHECK)

## Template matching criterias:

This section has the templates for matching criteria, that will be defined at the end of the rules. This include assessing criteria_met status for initial_decision_track, rhinosinusitis, acute_rhinosinusitis, bacterial_suspicion, sa_acute_ttm_need, chronic_rhinosinusitis, bacterial_complication, watchful_waiting, worsening_no_improvement, severe_infection_hospital, beta_lactam_allergy and clinical_improvement

In [1739]:
## patient initial decision track criteria_met status

DEFTEMPLATE_INITIAL_DECISION_TRACK = """
(deftemplate initial_decision_track
    (slot criteria_met (type SYMBOL)
        (allowed-symbols yes no unknown))
)
"""
env.build(DEFTEMPLATE_INITIAL_DECISION_TRACK)

## patient rhinosinusitis criteria_met status

DEFTEMPLATE_RHINOSINUSITIS= """
(deftemplate rhinosinusitis
    (slot criteria_met (type SYMBOL)
        (allowed-symbols yes no unknown))
)
"""
env.build(DEFTEMPLATE_RHINOSINUSITIS)

## patient rhinosinusitis duration (check if acute or not) criteria_met status

DEFTEMPLATE_ACUTE_RHINOSINUSITIS = """
(deftemplate acute_rhinosinusitis
    (slot criteria_met (type SYMBOL)
    (allowed-symbols yes no unknown))
)
"""
env.build(DEFTEMPLATE_ACUTE_RHINOSINUSITIS)

## patient case bacterial suspicion criteria_met status

DEFTEMPLATE_BACTERIAL_SUSPICION = """
(deftemplate bacterial_suspicion
    (slot criteria_met (type SYMBOL)
    (allowed-symbols yes no unknown))
)
"""
env.build(DEFTEMPLATE_BACTERIAL_SUSPICION)

## patient with subacute rhinosinusitis - whether in need of acute management criteria_met status

DEFTEMPLATE_SA_ACUTE_TTM_NEED = """
(deftemplate sa_acute_ttm_need
    (slot criteria_met (type SYMBOL)
    (allowed-symbols yes no unknown)))
"""
env.build(DEFTEMPLATE_SA_ACUTE_TTM_NEED)

## patient chronic rhinosinusitis check (chronic vs subacute) criteria_met status

DEFTEMPLATE_CHRONIC_RHINOSINUSITIS = """
(deftemplate chronic_rhinosinusitis
    (slot criteria_met (type SYMBOL)
    (allowed-symbols yes no unknown)))
"""
env.build(DEFTEMPLATE_CHRONIC_RHINOSINUSITIS)

## bacterial complication criteria_met status

DEFTEMPLATE_BACTERIAL_COMPLICATION = """
(deftemplate bacterial_complication
    (slot criteria_met (type SYMBOL)
    (allowed-symbols yes no unknown)))
"""
env.build(DEFTEMPLATE_BACTERIAL_COMPLICATION)


## bacterial watchful waiting eligibility criteria_met status

DEFTEMPLATE_WATCHFUL_WAITING = """
(deftemplate watchful_waiting
    (slot criteria_met (type SYMBOL)
    (allowed-symbols yes no unknown)))
"""
env.build(DEFTEMPLATE_WATCHFUL_WAITING)

## bacterial worsening for 7 days assessment criteria_met status

DEFTEMPLATE_WORSENING_NO_IMPROVEMENT = """
(deftemplate worsening_no_improvement
    (slot criteria_met (type SYMBOL)
    (allowed-symbols yes no unknown)))
"""
env.build(DEFTEMPLATE_WORSENING_NO_IMPROVEMENT)

## Severe Infection in need for Hospitalization criteria_met status

DEFTEMPLATE_SEVERE_INFECTION_HOSPITAL = """
(deftemplate severe_infection_hospital
    (slot criteria_met (type SYMBOL)
    (allowed-symbols yes no unknown)))
"""
env.build(DEFTEMPLATE_SEVERE_INFECTION_HOSPITAL)

## Beta-lactam allergy criteria_met status

DEFTEMPLATE_BETA_LACTAM_ALLERGY= """
(deftemplate beta_lactam_allergy
    (slot criteria_met (type SYMBOL)
    (allowed-symbols yes no unknown)))
"""
env.build(DEFTEMPLATE_BETA_LACTAM_ALLERGY)

## clinical improvement criteria_met status

DEFTEMPLATE_CLINICAL_IMPROVEMENT= """
(deftemplate clinical_improvement
    (slot criteria_met (type SYMBOL)
    (allowed-symbols yes no unknown)))
"""
env.build(DEFTEMPLATE_CLINICAL_IMPROVEMENT)


## Initialize Knowledgebase

Keeping criteria_met status as unknown initially, so that they change as the rules as being read.


In [1741]:
DEFFCATS_INITIAL_STATUS = """
(deffacts starting_facts "Set the facts to unknown"
    (initial_decision_track (criteria_met unknown))
    (rhinosinusitis (criteria_met unknown))
    (acute_rhinosinusitis (criteria_met unknown))
    (bacterial_suspicion (criteria_met unknown))
    (chronic_rhinosinusitis (criteria_met unknown))
    (sa_acute_ttm_need (criteria_met unknown))
    (bacterial_complication (criteria_met unknown))
    (watchful_waiting (criteria_met unknown))
    (worsening_no_improvement (criteria_met unknown))
    (severe_infection_hospital (criteria_met unknown))
    (beta_lactam_allergy (criteria_met unknown))
    (clinical_improvement (criteria_met unknown))
)
"""
env.build(DEFFCATS_INITIAL_STATUS)

env.reset() ## reseting the environment to make sure the deffacts are added

## Inference Rules

### Initial Decision Track Rules

The `defrules` in this section support gathering input to ensure that the clinician is correctly using the decision track. Ask about the age, sex, if female pregancy status, predominance of upper airway symptoms and clinican suspicion of rhinosinusitis. 

**Initial Decision Track Criteria**: 
For decision track to continue asking questions, the answers need to match ALL initial decision track criteria: 
    - patient => 18 years of age
    - predominance of upper airway symptoms
    - clinical suspicion for rhinosinusitis

if (initial_decision_track (criteria_met no)), no further information is collected and a message will be printed: " This decision track is for adult cases with suspicion for rhinosinusitis, not pediatric or cases without predominance of upper airway symptoms or suspicion for rhinosinusitis. If pediatric patient, please refer to pediatric guidelines as these might differ from the adult one. If predominant symptoms are not upper airway then other diagnosis need to be considered or if there is no rhinossinusitis suspicion, this is not the correct decision track for this case."

if (initial_decision_track (criteria_met yes)), a message will be printed: "Apart from using this decision track, also consider differencial diagnosis, ruling them out, such as dental pain (if unilateral facial pain), neuralgic atypical facial pain or trigeminal neuralgia, temporomandibular joint pain, migrain, rhinitis (allergic, eosinophilic nonallergic), vasomotor headaches, neoplastic conditions". And, the following rule will start (start_execution_rhinosinusitis yes).

String messages will be printed in the reporting rules below.

In [1744]:

## all decision track prompts
prompt_map = {
    "patient:age_is": "What is the patient's age (in years)?: ",
    "patient:sex_is": "What is the patient's sex (male or female)?: ",
    "pregnancy_status:preg_is": "Is she pregnant (yes or no or unknown)?: ",
    "upper_airway_predom:with_upper_airway_symp_predom": "Does the patient have predominance of upper airway symptoms (yes or no)?: ",
    "rhinosinusitis_suspicion:with_rhinosinusitis_suspicion" : "Based on clinical assessment, is there suspicion for rhinossinusitis (yes or no)?: ",
    "symptoms:purulent_nasal_discharge" : "Is there purulent nasal discharge (yes no)?: ",
    "major_symptoms:anterior_nasal_discharge" : "Is there purulent anterior nasal discharge (yes no)?: ",
    "major_symptoms:posterior_nasal_discharge" : "Is there purulent or clear posterior nasal discharge (yes no)?: ",
    "major_symptoms:nasal_congestion_obstruction" : "Is there nasal congestion or obstruction (yes no)?: ",
    "major_symptoms:facial_congestion_fullness" : "Is there facial congestion or fullness (yes no)?: ",
    "major_symptoms:facial_pain_pressure" : "Is there facial pain or pressure (yes no)?: ",
    "major_symptoms:hyposmia_anosmia" : "Is there hyposmia or anosmia (yes no)?: ",
    "major_symptoms:fever" : "Is there fever (yes no)?: ",
    "minor_symptoms:headache" : "Is there headache (yes no)?: ",
    "minor_symptoms:ear_pain_pressure_fullness" : "Is there ear pain, pressure and fullness (yes no)?: ",
    "minor_symptoms:halitosis" : "Is there halitosis (yes no)?: ",
    "minor_symptoms:dental_pain" : "Is there dental pain (yes no)?: ",
    "minor_symptoms:cough" : "Is there cough (yes no)?: ",
    "minor_symptoms:fatigue" : "Is there fatigue (yes no)?: ",
    "rhinosinusitis_duration:four_weeks_less":"Is the duration of this presentation four weeks or less (yes no)?:",
    "chronic_rhinosinusitis_duration:twelve_weeks_more":"Is the duration of this presentation twelve weeks or more (yes no)?:",
    "bacterial_suspicion_check:total_presentation_days":"How many days has it been since the beggining of this presentation(in days)?:",
    "bacterial_suspicion_check:improvement":"Has there been any improvement (yes no)?:",
    "bacterial_suspicion_check:progressive_worsening":"Has it been progresivelly worsening (yes no)?:",
    "bacterial_suspicion_check:double_sickening":"Was the patient improving at first but than started worsening again (yes no)?:",
    "bacterial_suspicion_check:severe_3_days": "Have the symptoms been severe for at least 3 (three) days (yes no):",
    "bacterial_suspicion_check:bacterial_infection":"Based on your clinical assessment,is there suspicion for bacterial infection(yes no)?:",
    "bacterial_complication_check:bact_comp":"Is there suspicion for complication (eg.: (peri)orbital/soft tissue cellulitis/abscess;intracranial meningitis/abscess) (yes no)?:",
    "subacute_ttm_need:acute_ttm":"Based on your clinical assessment,is there a need for acute management (yes no)?:",
    "watchful_waiting_check:watch_wait":"Based on your clinical assessment, is this patient eligible for watchful waiting (yes no)?: ",
    "worsening_no_improvement_check:worse_noimprove":"When reassessing after a week, is there worsening or no improvement (yes no)?: ",
    "severe_infection_hospital_check:severe_hospital":"Is this a severe case in need of hospitalization (yes no)?: ",
    "beta_lactam_allergy_check:allergy":"Is the patient allergic to beta-lactam antibiotics (yes no)?: ",
    "clinical_improvement_check:better":"Is there clinical improvement after initial treatment(yes no)?: ",
}
build_read_assert(env, prompt_map)

## Initial Decision Track rule

DEFRULE_READ_INITIAL_DECISION_TRACK = """
(defrule reading_input_initial
    =>
    (read_assert patient)
    (read_assert upper_airway_predom)
    (read_assert rhinosinusitis_suspicion))
"""
env.build(DEFRULE_READ_INITIAL_DECISION_TRACK)

## Check for pregnancy status if female

DEFRULE_INPUT_PREG_STATUS = """
(defrule reading_input_preg_status "Ask for pregnancy status"
        (patient(sex_is female))
        =>
        (read_assert pregnancy_status))
"""
env.build(DEFRULE_INPUT_PREG_STATUS)
                   
# ; RULE: Initial Decision Track Criteria Are NOT MET
# ; *Forward chaining to determine if this is the INCORRECT decision track based on initial decision track facts
# ; INPUT: Initial Decision Track Criteria
# ; OUTPUT: Incorrect Decision Track and string message

DEFRULE_INITIAL_DECISION_TRACK_INCORRECT = """
(defrule initial_decision_track_incorrect "Rule to define this decision track is not correct for this clinical case"
    (logical
        (patient (age_is ?age))
        (upper_airway_predom (with_upper_airway_symp_predom ?upper_airway_predom))
        (rhinosinusitis_suspicion (with_rhinosinusitis_suspicion ?RS_suspicion))
        (or
            (test (< ?age 18)) 
            (upper_airway_predom (with_upper_airway_symp_predom no))
            (rhinosinusitis_suspicion (with_rhinosinusitis_suspicion no))))
    
    
    ?f1 <-(initial_decision_track (criteria_met unknown))

    => 

    (modify ?f1 (criteria_met no)))
"""
env.build(DEFRULE_INITIAL_DECISION_TRACK_INCORRECT)

# ; RULE: Initial Decision Track Criteria Are MET
# ; *Forward chaining to determine if this is the CORRECT decision track based on initial decision track facts
# ; INPUT: Initial Decision Track Criteria
# ; OUTPUT: Correct Decision Track and string message

DEFRULE_INITIAL_DECISION_TRACK_CORRECT = """
(defrule initial_decision_track_correct "Rule to define this decision track is correct for this clinical case"
    (logical
        (patient (age_is ?age))
        (upper_airway_predom (with_upper_airway_symp_predom ?upper_airway_symptoms))
        (rhinosinusitis_suspicion (with_rhinosinusitis_suspicion ?RS_suspicion))
        (and 
        	(test (>= ?age 18))
            (upper_airway_predom (with_upper_airway_symp_predom yes))
            (rhinosinusitis_suspicion (with_rhinosinusitis_suspicion yes))))
    ?f1 <-(initial_decision_track (criteria_met unknown))

    => 

    (modify ?f1 (criteria_met yes)))
"""
env.build(DEFRULE_INITIAL_DECISION_TRACK_CORRECT)



### Initial Decision Track Check Reporting Rules

This section contains the reporting rules (with string messages), based on the answers for the initial_decision_track above.

In [1746]:
DEFRULE_DT_INCORRECT = """
(defrule decision_track_incorrect
    (initial_decision_track (criteria_met no))
    =>
    (println "___________")
    (println "
    This decision track is for adult cases with predominant upper aiway symptoms, that raised clinical suspicion for rhinosinusitis.
    If pediatric patient, please refer to pediatric guidelines as these differ from the adult guidelines. 
    If there is no rhinossinusitis suspicion, this is not the correct decision track for this case.
    If predominant symptoms are not upper airway then other diagnoses need to be considered.")
    (println "___________"))
"""
env.build(DEFRULE_DT_INCORRECT)

DEFRULE_DT_CORRECT = """
(defrule decision_track_correct
    (initial_decision_track (criteria_met yes))
    =>
    (println "___________")
    (println "
    This decision track is adequate for this case.
    Apart from using this decision track, also consider differencial diagnosis, ruling them out, such as:
    dental pain (if unilateral facial pain), neuralgic atypical facial pain or trigeminal neuralgia, 
    temporomandibular joint pain, migrain, rhinitis (allergic, eosinophilic nonallergic), 
    vasomotor headaches, neoplastic conditions. ")
    (println "___________")
    (assert (start_rhinosinusitis yes)))
"""
env.build(DEFRULE_DT_CORRECT)


### Rhinosinusitis Rules
The `defrules` in this section support gathering input to ensure the case matches rhinosinusitis criteria, asking about rhinosinusitis symptoms. The symptoms are classified as following:

SYMPTOMS:
- purulent nasal discharge
- 
MAJOR SYMPTOMS:
- purulent anterior nasal discharge
- purulent/clear posterior
- nasal congestion or obstruction
- facial congestion or fullness
- facial pain or pressure
- hyposmia or anosmia
- fever
  
MINOR SYMPTOMS:
- Headache
- Ear pain, pressure, fullness
- Halitosis
- dental pain
- Cough
- Fatigue

The criteria will be the following:

- AAO-HNS: (purulent nasal discharge AND (nasal obstruction OR (facial pain or pressure OR facial congestion or fullness)))

OR 

- IDSA criteria: (2 major OR (1 major AND => 2 minor symptoms)))

if rhinosinusitis criteria is NOT MET, stop execution and print string: "This patient does not match the criteria for rhinosinusitis. Manage as a viral upper airway infection/acute rhinopharingitis."

if rhinosinusitis criteria is MET, start rhinosinusitis duration assessment and print string: "This patient matches the criteria for rhinosinusitis."


In [1748]:
DEFRULE_READ_INPUT_RHINOSINUSITIS = """
(defrule reading-input-rhinosinusitis
    (initial_decision_track (criteria_met yes))
    (start_rhinosinusitis yes)
    (rhinosinusitis (criteria_met unknown))
    (acute_rhinosinusitis (criteria_met unknown))
    (bacterial_suspicion (criteria_met unknown))
    (sa_acute_ttm_need (criteria_met unknown))
    (chronic_rhinosinusitis (criteria_met unknown))
    =>
    (read_assert symptoms)
    (read_assert major_symptoms)
    (read_assert minor_symptoms))
"""
env.build(DEFRULE_READ_INPUT_RHINOSINUSITIS)


# ; RULE: Rhinosinusitis Criteria not Meet
# ; *Forward chaining to determine if this case does not match the criteria for rhinosinusitis based on symptoms
# ; INPUT:  Symptoms
# ; OUTPUT: rhinosinusitis criteria not met and string message
DEFRULE_RHINOSINUSITIS_CRITERIA_NOT_MET = """
(defrule rhinosinusitis_criteria_not_met "Rule to define if this case does not match the criteria for rhinosinusitis"
    (logical
        (initial_decision_track (criteria_met yes))
        (symptoms (purulent_nasal_discharge ?purulent_nasal_discharge))
        (major_symptoms (nasal_congestion_obstruction ?nasal_congestion_obstruction))
        (major_symptoms (facial_pain_pressure ?facial_pain_pressure))
        (major_symptoms (facial_congestion_fullness ?facial_congestion_fullness))
        (major_symptoms (anterior_nasal_discharge ?anterior_nasal_discharge))
        (major_symptoms (posterior_nasal_discharge ?posterior_nasal_discharge))
        (major_symptoms (nasal_congestion_obstruction ?nasal_congestion_obstruction))
        (major_symptoms (facial_pain_pressure ?facial_pain_pressure))
        (major_symptoms (facial_congestion_fullness ?facial_congestion_fullness))
        (major_symptoms (hyposmia_anosmia ?hyposmia_anosmia))
        (major_symptoms (fever ?fever))
        (minor_symptoms (headache ?headache))
        (minor_symptoms (ear_pain_pressure_fullness ?ear_pain_pressure_fullness))
        (minor_symptoms (halitosis ?halitosis))
        (minor_symptoms (dental_pain ?dental_pain))
        (minor_symptoms (cough ?cough))
        (minor_symptoms (fatigue ?fatigue))

        (test 
            (not
                (or
                    (and
                        (eq ?purulent_nasal_discharge yes)
                        (or
                            (eq ?nasal_congestion_obstruction yes)
                            (eq ?facial_pain_pressure yes)
                            (eq ?facial_congestion_fullness yes)))
                    (and
                        (or
                            (eq ?anterior_nasal_discharge yes)
                            (eq ?posterior_nasal_discharge yes)
                            (eq ?nasal_congestion_obstruction yes)
                            (eq ?facial_pain_pressure yes)
                            (eq ?facial_congestion_fullness yes)
                            (eq ?hyposmia_anosmia yes)
                            (eq ?fever yes))
                        (or
                            (neq ?anterior_nasal_discharge yes)
                            (neq ?posterior_nasal_discharge yes)
                            (neq ?nasal_congestion_obstruction yes)
                            (neq ?facial_pain_pressure yes)
                            (neq ?facial_congestion_fullness yes)
                            (neq ?hyposmia_anosmia yes)
                            (neq ?fever yes)))

                    (and
                        (or
                            (eq ?anterior_nasal_discharge yes)
                            (eq ?posterior_nasal_discharge yes)
                            (eq ?nasal_congestion_obstruction yes)
                            (eq ?facial_pain_pressure yes)
                            (eq ?facial_congestion_fullness yes)
                            (eq ?hyposmia_anosmia yes)
                            (eq ?fever yes))
                        (and
                            (or
                                (eq ?headache yes)
                                (eq ?ear_pain_pressure_fullness yes)
                                (eq ?halitosis yes)
                                (eq ?dental_pain yes)
                                (eq ?cough yes)
                                (eq ?fatigue yes))
                            (or
                                (neq ?headache yes)
                                (neq ?ear_pain_pressure_fullness yes)
                                (neq ?halitosis yes)
                                (neq ?dental_pain yes)
                                (neq ?cough yes)
                                (neq ?fatigue yes))))))))
                            
    ?f1 <- (rhinosinusitis (criteria_met unknown))
    =>
    (modify ?f1 (criteria_met no))
    (println "This patient does NOT match the criteria for rhinosinusitis. Manage as a viral upper airway infection/acute rhinopharyngitis."))

"""
env.build(DEFRULE_RHINOSINUSITIS_CRITERIA_NOT_MET)


# ; RULE: Rhinosinusitis Criteria Meet
# ; *Forward chaining to determine if this case matches the criteria for rhinosinusitis based on symptoms
# ; INPUT:  Symptoms
# ; OUTPUT: rhinosinusitis criteria met and string message and assert start rhinosinusitis duration 
DEFRULE_RHINOSINUSITIS_CRITERIA_MET = """
(defrule rhinosinusitis_criteria_met "Rule to define if this case matches the criteria for rhinosinusitis"
    (logical
        (initial_decision_track (criteria_met yes))
        (symptoms (purulent_nasal_discharge ?purulent_nasal_discharge))
        (major_symptoms (nasal_congestion_obstruction ?nasal_congestion_obstruction))
        (major_symptoms (facial_pain_pressure ?facial_pain_pressure))
        (major_symptoms (facial_congestion_fullness ?facial_congestion_fullness))
        (major_symptoms (anterior_nasal_discharge ?anterior_nasal_discharge))
        (major_symptoms (posterior_nasal_discharge ?posterior_nasal_discharge))
        (major_symptoms (nasal_congestion_obstruction ?nasal_congestion_obstruction))
        (major_symptoms (facial_pain_pressure ?facial_pain_pressure))
        (major_symptoms (facial_congestion_fullness ?facial_congestion_fullness))
        (major_symptoms (hyposmia_anosmia ?hyposmia_anosmia))
        (major_symptoms (fever ?fever))
        (minor_symptoms (headache ?headache))
        (minor_symptoms (ear_pain_pressure_fullness ?ear_pain_pressure_fullness))
        (minor_symptoms (halitosis ?halitosis))
        (minor_symptoms (dental_pain ?dental_pain))
        (minor_symptoms (cough ?cough))
        (minor_symptoms (fatigue ?fatigue))
        

        (test 
            (or
                (and
                    (eq ?purulent_nasal_discharge yes)
                    (or
                        (eq ?nasal_congestion_obstruction yes)
                        (eq ?facial_pain_pressure yes)
                        (eq ?facial_congestion_fullness yes)))
                (and
                    (or
                        (eq ?anterior_nasal_discharge yes)
                        (eq ?posterior_nasal_discharge yes)
                        (eq ?nasal_congestion_obstruction yes)
                        (eq ?facial_pain_pressure yes)
                        (eq ?facial_congestion_fullness yes)
                        (eq ?hyposmia_anosmia yes)
                        (eq ?fever yes))
                    (or
                        (neq ?anterior_nasal_discharge yes)
                        (neq ?posterior_nasal_discharge yes)
                        (neq ?nasal_congestion_obstruction yes)
                        (neq ?facial_pain_pressure yes)
                        (neq ?facial_congestion_fullness yes)
                        (neq ?hyposmia_anosmia yes)
                        (neq ?fever yes)))

                (and
                    (or
                        (eq ?anterior_nasal_discharge yes)
                        (eq ?posterior_nasal_discharge yes)
                        (eq ?nasal_congestion_obstruction yes)
                        (eq ?facial_pain_pressure yes)
                        (eq ?facial_congestion_fullness yes)
                        (eq ?hyposmia_anosmia yes)
                        (eq ?fever yes))
                    (and
                        (or
                            (eq ?headache yes)
                            (eq ?ear_pain_pressure_fullness yes)
                            (eq ?halitosis yes)
                            (eq ?dental_pain yes)
                            (eq ?cough yes)
                            (eq ?fatigue yes))
                        (or
                            (neq ?headache yes)
                            (neq ?ear_pain_pressure_fullness yes)
                            (neq ?halitosis yes)
                            (neq ?dental_pain yes)
                            (neq ?cough yes)
                            (neq ?fatigue yes)))))))
    
    
    ?f1 <-(rhinosinusitis (criteria_met unknown))

    => 

    (modify ?f1 (criteria_met yes))
    (assert (start_rhinosinusitis_duration yes)) 
    (println "This patient matches the criteria for rhinosinusitis."))
"""
env.build(DEFRULE_RHINOSINUSITIS_CRITERIA_MET)



### Rhinosinusitis Duration (Acute vs Non-acute)

The `defrules` in this section support gathering input on the duration of the symptoms to assess if the case is acute or not, asking about rhinosinusitis symptoms duration.

Both the IDSA and AAO-HNS guidelines consider a rhinosinusitis case acute when the duration of the symptoms is 4 weeks or less. 

If NOT ACUTE, it will (assert (start_chronic_rhinosinusitis_duration yes).

If ACUTE, it will (assert (start_bacterial_suspicion yes)).

In [1750]:
DEFRULE_READ_RHINOSINUSITIS_DURATION = """
(defrule reading_input_rhinosinusitis_duration
    (initial_decision_track (criteria_met yes))
    (start_rhinosinusitis yes)
    (rhinosinusitis (criteria_met yes))
    (start_rhinosinusitis_duration yes)
    =>
    (read_assert rhinosinusitis_duration))
"""
env.build(DEFRULE_READ_RHINOSINUSITIS_DURATION)


# ; RULE: Rhinosinusitis Duration not acute
# ; *Forward chaining to determine if this case is not acute
# ; INPUT:  rhinosinusitis_duration
# ; OUTPUT: not acute rhinosinusitis and assert start chronic rhinosinusitis duration
DEFRULE_RHINOSINUSITIS_DURATION_NONACUTE = """
(defrule rhinosinusitis_duration_non_acute "Rule to define if not acute rhinosinusitis "
    (logical
        (initial_decision_track (criteria_met yes))
        (rhinosinusitis (criteria_met yes))
        (rhinosinusitis_duration (four_weeks_less ?four_weeks_less))
        
        (rhinosinusitis_duration (four_weeks_less no)))
    
    ?f1 <-(acute_rhinosinusitis (criteria_met unknown))

    => 

    (modify ?f1 (criteria_met no))
    (assert (start_chronic_rhinosinusitis_duration yes)))
"""
env.build(DEFRULE_RHINOSINUSITIS_DURATION_NONACUTE)


# ; RULE: Rhinosinusitis Duration Acute
# ; *Forward chaining to determine if this case is acute
# ; INPUT:  rhinosinusitis_duration
# ; OUTPUT: acute rhinosinusitis and assert start bacterial suspicion
DEFRULE_RHINOSINUSITIS_DURATION_ACUTE = """
(defrule rhinosinusitis_duration_acute "Rule to define if acute rhinosinusitis "
    (logical
        (initial_decision_track (criteria_met yes))
        (rhinosinusitis (criteria_met yes))
        (rhinosinusitis_duration (four_weeks_less ?four_weeks_less))
     
        (rhinosinusitis_duration (four_weeks_less yes)))
    
    ?f1 <-(acute_rhinosinusitis (criteria_met unknown))

    =>

    (modify ?f1 (criteria_met yes))
    (assert (start_bacterial_suspicion yes))) 

"""
env.build(DEFRULE_RHINOSINUSITIS_DURATION_ACUTE)



### Acute Rhinosinusitis Bacterial Suspicion

The `defrules` in this section support gathering input on characteristics that raise suspicion for bacterial infection in acute cases.

If any of the characteristics, 10 or more days of presentation, not improving, progresivelly worsening, double sickening, severe symptoms for 3 or more days, or if the clinicians considered the case as suspicious for bacterial infection, then will consider the case as suspicious for bacterial infection: 

If bacterial_suspicion yes: print "Diagnosis: acute bacterial rhinosinusitis" and (assert (start_bacterial_complications yes))) 

If bacterial_suspicion no: print " Diagnosis acute viral rhinosinusitis. Prescribe symptomatics(analgesics, antipiretics) and nasal irrigation with 0.9% saline. Consider intranasal corticosteroids for symptomatic relief. Always remember to rule out the differential diagnsoses. Reassess in case of worsening of no improvement.


In [1752]:
DEFRULE_READ_BACTERIAL_SUSPICION_CHECK = """
(defrule reading_input_bacterial_suspicion_check
    (initial_decision_track (criteria_met yes))
    (start_rhinosinusitis yes)
    (rhinosinusitis (criteria_met yes))
    (start_rhinosinusitis_duration yes)
    (acute_rhinosinusitis (criteria_met yes))
    (start_bacterial_suspicion yes)
    =>
    (read_assert bacterial_suspicion_check))
"""
env.build(DEFRULE_READ_BACTERIAL_SUSPICION_CHECK)

# ; RULE: Acute Rhinosinusitis With NO Bacterial Suspicion
# ; *Forward chaining to determine if NO bacterial suspicion
# ; INPUT:  bacterial_suspicion
# ; OUTPUT: NOT suspicious of bacterial infection and print message
DEFRULE_BACTERIAL_SUSPICION_NOT_MET = """
(defrule bacterial_suspicion_not_met "Rule to define acute rhinosinusitis with bacterial suspicion NOT met "
    (logical
        (initial_decision_track (criteria_met yes))
        (rhinosinusitis (criteria_met yes))
        (acute_rhinosinusitis (criteria_met yes))
        (bacterial_suspicion_check  (total_presentation_days ?days))
        (bacterial_suspicion_check  (improvement ?improv))
        (bacterial_suspicion_check  (progressive_worsening ?prog_worse))
        (bacterial_suspicion_check  (double_sickening ?doub_sick))
        (bacterial_suspicion_check (severe_3_days ?sev_days))
        (bacterial_suspicion_check  (bacterial_infection ?bact_susp))
     
        (and
            (test (not(>= ?days 10)))
            (bacterial_suspicion_check  (improvement yes))
            (bacterial_suspicion_check  (progressive_worsening no))
            (bacterial_suspicion_check  (double_sickening no))
            (bacterial_suspicion_check  (severe_3_days no))
            (bacterial_suspicion_check  (bacterial_infection no))))
    
    ?f1 <-(bacterial_suspicion (criteria_met unknown))
    => 
    (modify ?f1 (criteria_met no))
    (println "Diagnosis: acute viral rhinosinusitis.Prescribe symptomatics (analgesics, antipiretics) and nasal irrigation with 0.9% saline. Consider intranasal corticosteroids for symptomatic relief"))
"""
env.build(DEFRULE_BACTERIAL_SUSPICION_NOT_MET)


# ; RULE: Acute Rhinosinusitis With Bacterial Suspicion
# ; *Forward chaining to determine if bacterial suspicion
# ; INPUT:  bacterial_suspicion
# ; OUTPUT: suspicious of bacterial infection and print message and start bacterial complications
DEFRULE_BACTERIAL_SUSPICION_MET = """
(defrule bacterial_suspicion_met "Rule to define acute rhinosinusitis with bacterial suspicion met "
    (logical
        (initial_decision_track (criteria_met yes))
        (rhinosinusitis (criteria_met yes))
        (acute_rhinosinusitis (criteria_met yes))
        (bacterial_suspicion_check  (total_presentation_days ?days))
        (bacterial_suspicion_check  (improvement ?improv))
        (bacterial_suspicion_check  (progressive_worsening ?prog_worse))
        (bacterial_suspicion_check  (double_sickening ?doub_sick))
        (bacterial_suspicion_check (severe_3_days ?sev_days))
        (bacterial_suspicion_check  (bacterial_infection ?bact_susp))
     
        (or
            (test (>= ?days 10))
            (bacterial_suspicion_check  (improvement no))
            (bacterial_suspicion_check  (progressive_worsening yes))
            (bacterial_suspicion_check  (double_sickening yes))
            (bacterial_suspicion_check  (severe_3_days yes))
            (bacterial_suspicion_check  (bacterial_infection yes))))
    
    ?f1 <-(bacterial_suspicion (criteria_met unknown))

    => 

    (modify ?f1 (criteria_met yes))
    (println " Diagnosis: Acute Bacterial Rhinosinusitis. ") 
    (assert (start_bacterial_complications yes))) 

"""
env.build(DEFRULE_BACTERIAL_SUSPICION_MET)

### Non-Acute Rhinosinusitis Duration

The `defrules` in this section support gathering input on the duration (if 12 weeks or more) to check whether it is subacute or chronic.

If duration of 12 weeks or more, it will print message: "Prescribe nasal irrigation with 0.9% saline. Consider intranasal corticosteroids for symptomatic relief.Refer to ENT to assess for potential chronic rhinosinusitis, poliposis, invasive fungal or allergic fungal rhinosinusitis;
If recurrent, assess for cystic fibrosis, immotile cilia disorders, ciliary dyskinesia, immune deficiency, prior history of sinus surgery, and anatomic abnormalities (eg, deviated nasal septum)".

If lower than 12 weeks (subacute), it will print message: "Diagnosis: Subacte Rhinosinusitis" and (assert (start_subacute_ttm_need yes))

In [1754]:
DEFRULE_READ_CHRONIC_RHINOSINUSITIS_DURATION = """
(defrule reading_input_chronic_rhinosinusitis_duration
    (initial_decision_track (criteria_met yes))
    (start_rhinosinusitis yes)
    (rhinosinusitis (criteria_met yes))
    (start_rhinosinusitis_duration yes)
    (acute_rhinosinusitis (criteria_met no))
    (start_chronic_rhinosinusitis_duration yes)
    =>
    (read_assert chronic_rhinosinusitis_duration))
"""
env.build(DEFRULE_READ_CHRONIC_RHINOSINUSITIS_DURATION)


# ; RULE: Chronic Rhinosinusitis Duration NOT met
# ; *Forward chaining to determine if not chronic rhinosinusitis
# ; INPUT:  whether symptoms are present for twelve weeks or more or NOT
# ; OUTPUT: Subacute rhinosinusitis and print message
DEFRULE_CHRONIC_RHINOSINUSITIS_NOT_MET = """
(defrule chronic_rhinosinusitis_not_met "Rule to define if does NOT match chronic rhinosinusitis duration "
    (logical
        (initial_decision_track (criteria_met yes))
        (rhinosinusitis (criteria_met yes))
        (acute_rhinosinusitis (criteria_met no))
        (chronic_rhinosinusitis_duration (twelve_weeks_more ?chronic_duration))
     
        (chronic_rhinosinusitis_duration (twelve_weeks_more no)))
    
    ?f1 <-(chronic_rhinosinusitis (criteria_met unknown))

    => 

    (modify ?f1 (criteria_met no))
    (println " Diagnosis: Subacute rhinosinusitis")
    (assert (start_subacute_ttm_need yes)))
"""
env.build(DEFRULE_CHRONIC_RHINOSINUSITIS_NOT_MET)

# ; RULE: Chronic Rhinosinusitis Duration Met
# ; *Forward chaining to determine if chronic rhinosinusitis
# ; INPUT:  whether symptoms are present for twelve weeks or more
# ; OUTPUT: Evaluate for chronic rhinosinusitis and print message
DEFRULE_CHRONIC_RHINOSINUSITIS_MET = """
(defrule chronic_rhinosinusitis_met "Rule to define if does match chronic rhinosinusitis duration "
    (logical
        (initial_decision_track (criteria_met yes))
        (rhinosinusitis (criteria_met yes))
        (acute_rhinosinusitis (criteria_met no))
        (chronic_rhinosinusitis_duration (twelve_weeks_more ?chronic_duration))
     
        (chronic_rhinosinusitis_duration (twelve_weeks_more yes)))
    
    ?f1 <-(chronic_rhinosinusitis (criteria_met unknown))

    => 

    (modify ?f1 (criteria_met yes))
    (println "Prescribe nasal irrigation with 0.9% saline.")
    (println "Consider intranasal corticosteroids for symptomatic relief.")
    (println "Refer to ENT to assess for potential chronic rhinosinusitis, poliposis,invasive fungal or allergic fungal rhinosinusitis;")
    (println "If recurrent, assess for cystic fibrosis, immotile cilia disorders, ciliary dyskinesia, immune deficiency, prior history of sinus surgery, and anatomic abnormalities (eg, deviated nasal septum).")
)
"""
env.build(DEFRULE_CHRONIC_RHINOSINUSITIS_MET)



### Subacute Acute Management Need Assessment

The `defrules` in this section support gathering input on the clinician's assessment of whether a subacute case requires acute management.

If NO need for acute management, print: "Prescribe nasal irrigation with 0.9% saline. Consider intranasal corticosteroids for symptomatic relief. Always remember to rule out the differential diagnsoses. Reassess in case of worsening of no improvement."

If NEED for acute management: (assert (start_bacterial_suspicion yes))

In [1756]:
DEFRULE_READ_SA_TTM_NEED= """
(defrule reading_input_sa_ttm_need
    (chronic_rhinosinusitis (criteria_met no))
    (start_subacute_ttm_need yes)
    =>
    (read_assert subacute_ttm_need))
"""
env.build(DEFRULE_READ_SA_TTM_NEED)    


# ; RULE: SA Rhinosinusitis NO acute treatment need
# ; *Forward chaining to determine if no acute treatment need for subacute case
# ; INPUT:  whether clinician considers that subacute case will require acute management
# ; OUTPUT: Subacute rhinosinusitis and print message
DEFRULE_SA_ACUTE_TTM_NEED_NO= """
(defrule sa_acute_ttm_need_no "Rule to define whether clinician considers that subacute case will require acute management (NOT) "
    (logical
        (chronic_rhinosinusitis (criteria_met no))
        (subacute_ttm_need (acute_ttm ?ttm_need))
     
        (subacute_ttm_need (acute_ttm no)))
    
    ?f1 <-(sa_acute_ttm_need (criteria_met unknown))

    => 

    (modify ?f1 (criteria_met no))
    (println " Prescribe nasal irrigation with 0.9% saline. Consider intranasal corticosteroids for symptomatic relief. 
    Always remember to rule out the differential diagnsoses. Reassess in case of worsening of no improvement."))
"""
env.build(DEFRULE_SA_ACUTE_TTM_NEED_NO)


# ; RULE: SA Rhinosinusitis acute treatment need
# ; *Forward chaining to determine if acute treatment need for subacute case
# ; INPUT:  whether clinician considers that subacute case will require acute management
# ; OUTPUT: start from bacterial infection suspicion
DEFRULE_SA_ACUTE_TTM_NEED_YES= """
(defrule sa_acute_ttm_need_yes "Rule to define whether clinician considers that subacute case will require acute management "
    (logical
        (chronic_rhinosinusitis (criteria_met no))
        (subacute_ttm_need (acute_ttm ?ttm_need))
     
        (subacute_ttm_need (acute_ttm yes)))
    
    ?f1 <-(sa_acute_ttm_need (criteria_met unknown))

    => 

    (modify ?f1 (criteria_met yes))
    (assert (start_bacterial_suspicion_2 yes)))
"""
env.build(DEFRULE_SA_ACUTE_TTM_NEED_YES)

### Subacute Rhinosinusitis Bacterial Suspicion

The `defrules` in this section support gathering input on characteristics that raise suspicion for bacterial infection in subacute cases.

If any of the characteristics, not improving, progresivelly worsening, double sickening, severe symptoms for 3 or more days, or if the clinicians considered the case as suspicious for bacterial infection, then will consider the case as suspicious for bacterial infection: 

If bacterial_suspicion yes: print "Diagnosis: Subacute bacterial rhinosinusitis" and (assert (start_bacterial_complications yes))) 

If bacterial_suspicion no: print " Diagnosis subacute viral rhinosinusitis. Prescribe symptomatics(analgesics, antipiretics) and nasal irrigation with 0.9% saline. Consider intranasal corticosteroids for symptomatic relief. Always remember to rule out the differential diagnsoses. Reassess in case of worsening of no improvement.


In [1758]:
DEFRULE_READ_BACTERIAL_SUSPICION_EVAL_2 = """
(defrule reading_input_bacterial_suspicion_eval_2
    (sa_acute_ttm_need (criteria_met yes))
    (start_bacterial_suspicion_2 yes)
    =>
    (read_assert bacterial_suspicion_check))
"""
env.build(DEFRULE_READ_BACTERIAL_SUSPICION_EVAL_2)

# ; RULE: Subacute Rhinosinusitis With NO Bacterial Suspicion
# ; *Forward chaining to determine if NO bacterial suspicion
# ; INPUT:  bacterial_suspicion
# ; OUTPUT: NOT suspicious of bacterial infection and print message
DEFRULE_NO_BACTERIAL_SUSPICION_2 = """
(defrule no_bacterial_suspicion_2 "Rule to define subacute rhinosinusitis with bacterial suspicion NOT met "
    (logical
        (sa_acute_ttm_need (criteria_met yes))
        (bacterial_suspicion_check  (improvement ?improv))
        (bacterial_suspicion_check  (progressive_worsening ?prog_worse))
        (bacterial_suspicion_check  (double_sickening ?doub_sick))
        (bacterial_suspicion_check (severe_3_days ?sev_days))
        (bacterial_suspicion_check  (bacterial_infection ?bact_susp))
     
        (and
            (bacterial_suspicion_check  (improvement yes))
            (bacterial_suspicion_check  (progressive_worsening no))
            (bacterial_suspicion_check  (double_sickening no))
            (bacterial_suspicion_check  (severe_3_days no))
            (bacterial_suspicion_check  (bacterial_infection no))))
    
    ?f1 <-(bacterial_suspicion (criteria_met unknown))
    => 
    (modify ?f1 (criteria_met no))
    (println "Diagnosis: Subacute viral rhinosinusitis.Prescribe symptomatics (analgesics, antipiretics) and nasal irrigation with 0.9% saline. Consider intranasal corticosteroids for symptomatic relief"))
"""
env.build(DEFRULE_NO_BACTERIAL_SUSPICION_2)


# ; RULE: Subacute Rhinosinusitis With Bacterial Suspicion
# ; *Forward chaining to determine if bacterial suspicion
# ; INPUT:  bacterial_suspicion
# ; OUTPUT: suspicious of bacterial infection and print message and start bacterial complications
DEFRULE_BACTERIAL_SUSPICION_YES_2= """
(defrule bacterial_suspicion_yes_2 "Rule to define subacute rhinosinusitis with bacterial suspicion met "
    (logical
        (sa_acute_ttm_need (criteria_met yes))
        (bacterial_suspicion_check  (improvement ?improv))
        (bacterial_suspicion_check  (progressive_worsening ?prog_worse))
        (bacterial_suspicion_check  (double_sickening ?doub_sick))
        (bacterial_suspicion_check (severe_3_days ?sev_days))
        (bacterial_suspicion_check  (bacterial_infection ?bact_susp))
     
        (or
            (bacterial_suspicion_check  (improvement no))
            (bacterial_suspicion_check  (progressive_worsening yes))
            (bacterial_suspicion_check  (double_sickening yes))
            (bacterial_suspicion_check  (severe_3_days yes))
            (bacterial_suspicion_check  (bacterial_infection yes))))
    
    ?f1 <-(bacterial_suspicion (criteria_met unknown))

    => 

    (modify ?f1 (criteria_met yes))
    (println " Diagnosis: Subacute Bacterial Rhinosinusitis. ") 
    (assert (start_bacterial_complications_2 yes))) 

"""
env.build(DEFRULE_BACTERIAL_SUSPICION_YES_2)

### Rule: Bacterial Rhinosinusitis Complications
The defrules in this section support gathering input on whether there is suspicion for complications.

If bacterial_complications_check yes: print "Consider the need for hospitalization. Order imaging exam and manage accordingly" and (assert (start_bacterial_complications yes)).

If bacterial_complications_check no: (assert (start_watch_wait yes))  and print "Prescribe symptomatics (analgesics, antipiretics) and nasal irrigation with 0.9% saline. Consider intranasal corticosteroids for symptomatic relief".


In [1760]:
DEFRULE_READ_BACTERIAL_COMPLICATION_CHECK = """
(defrule reading_input_bacterial_complication_check
    (initial_decision_track (criteria_met yes))
    (start_rhinosinusitis yes)
    (rhinosinusitis (criteria_met yes))
    (start_rhinosinusitis_duration yes)
    (acute_rhinosinusitis (criteria_met yes))
    (start_bacterial_suspicion yes)
    (bacterial_suspicion (criteria_met yes))
    (start_bacterial_complications yes)
    =>
    (read_assert bacterial_complication_check))
"""
env.build(DEFRULE_READ_BACTERIAL_COMPLICATION_CHECK)


# ; RULE: Acute Rhinosinusitis With Suspicion for Complications
# ; *Forward chaining to determine if complication suspicion
# ; INPUT:  bacterial_complication_check
# ; OUTPUT: suspicious of complications and print message
DEFRULE_BACTERIAL_COMPLICATION_CHECK_YES = """
(defrule bacterial_complication_check_yes "Rule to define if there is bacterial complication"
    (logical
        (initial_decision_track (criteria_met yes))
        (rhinosinusitis (criteria_met yes))
        (acute_rhinosinusitis (criteria_met yes))
        (bacterial_suspicion (criteria_met yes))
        (bacterial_complication_check (bact_comp ?complications))
        
        (bacterial_complication_check (bact_comp  yes)))

    
    ?f1 <-(bacterial_complication (criteria_met unknown))
    => 
    (modify ?f1 (criteria_met yes))
    (println "Consider the need for hospitalization. Order imaging exam and manage accordingly"))
"""
env.build(DEFRULE_BACTERIAL_COMPLICATION_CHECK_YES)


# ; RULE: Acute Rhinosinusitis With NO Suspicion for Complications
# ; *Forward chaining to determine if NO complication suspicion
# ; INPUT:  bacterial_complication_check
# ; OUTPUT: NOT suspicious of complications and print message
DEFRULE_BACTERIAL_COMPLICATION_CHECK_NO = """
(defrule bacterial_complication_check_no "Rule to define if there is NO bacterial complication"
    (logical
        (initial_decision_track (criteria_met yes))
        (rhinosinusitis (criteria_met yes))
        (acute_rhinosinusitis (criteria_met yes))
        (bacterial_suspicion (criteria_met yes))
        (bacterial_complication_check (bact_comp ?complications))
        
        (bacterial_complication_check (bact_comp  no)))

    
    ?f1 <-(bacterial_complication (criteria_met unknown))
    => 
    (modify ?f1 (criteria_met no))
    (println "Prescribe symptomatics (analgesics, antipiretics) and nasal irrigation with 0.9% saline. Consider intranasal corticosteroids for symptomatic relief")
    (assert (start_watch_wait yes))) 
"""
env.build(DEFRULE_BACTERIAL_COMPLICATION_CHECK_NO)


In [1761]:
DEFRULE_READ_BACTERIAL_COMPLICATION_EVAL_2 = """
(defrule reading_input_bacterial_complication_eval_2
    (sa_acute_ttm_need (criteria_met yes))
    (start_bacterial_suspicion_2 yes)
    (bacterial_suspicion (criteria_met yes))
    (start_bacterial_complications_2 yes)
    =>
    (read_assert bacterial_complication_check))
"""
env.build(DEFRULE_READ_BACTERIAL_COMPLICATION_EVAL_2)


# ; RULE: Acute Rhinosinusitis With Suspicion for Complications
# ; *Forward chaining to determine if complication suspicion
# ; INPUT:  bacterial_complication_check
# ; OUTPUT: suspicious of complications and print message
DEFRULE_BACTERIAL_COMPLICATION_EVAL_YES_2 = """
(defrule bacterial_complication_eval_yes_2 "Rule to define if there is bacterial complication"
    (logical
        (sa_acute_ttm_need (criteria_met yes))
        (bacterial_suspicion (criteria_met yes))
        (bacterial_complication_check (bact_comp ?complications))
        
        (bacterial_complication_check (bact_comp  yes)))

    
    ?f1 <-(bacterial_complication (criteria_met unknown))
    => 
    (modify ?f1 (criteria_met yes))
    (println "Consider the need for hospitalization. Order imaging exam and manage accordingly"))
"""
env.build(DEFRULE_BACTERIAL_COMPLICATION_EVAL_YES_2)


# ; RULE: Acute Rhinosinusitis With NO Suspicion for Complications
# ; *Forward chaining to determine if NO complication suspicion
# ; INPUT:  bacterial_complication_check
# ; OUTPUT: NOT suspicious of complications and print message
DEFRULE_BACTERIAL_COMPLICATION_EVAL_NO_2 = """
(defrule bacterial_complication_eval_no_2 "Rule to define if there is NO bacterial complication"
    (logical
        (sa_acute_ttm_need (criteria_met yes))
        (bacterial_suspicion (criteria_met yes))
        (bacterial_complication_check (bact_comp ?complications))
        
        (bacterial_complication_check (bact_comp  no)))

    
    ?f1 <-(bacterial_complication (criteria_met unknown))
    => 
    (modify ?f1 (criteria_met no))
    (println "Prescribe symptomatics (analgesics, antipiretics) and nasal irrigation with 0.9% saline. Consider intranasal corticosteroids for symptomatic relief")
    (assert (start_watch_wait yes))) 
"""
env.build(DEFRULE_BACTERIAL_COMPLICATION_EVAL_NO_2)

### Rule: Watchful Waiting Eligibility

The defrules in this section support gathering input on whether there is eligibility for watchful waiting.

If there is eligibility: (assert (start_worsening_no_improvement_check yes)) and print message: "Reassess in 7 days"
If there is NO eligibility: (assert (start_severe_infection_hospital_check yes))

In [1763]:
DEFRULE_READ_WATCHFUL_WAITING_CHECK = """
(defrule reading_input_watchful_waiting_check
    (bacterial_suspicion (criteria_met yes))
    (bacterial_complication (criteria_met no))
    (start_watch_wait yes)
    =>
    (read_assert watchful_waiting_check))
"""
env.build(DEFRULE_READ_WATCHFUL_WAITING_CHECK)


# ; RULE: Eligibility for watchful waiting
# ; *Forward chaining to determine if eligible for watchful waiting.
# ; INPUT:  watchful_waiting_check
# ; OUTPUT: (assert (start_worsening_no_improvement_check yes))
DEFRULE_WATCHFUL_WAITING_CHECK_YES = """
(defrule watchful_waiting_check_yes "Rule to define if there is eligibility for watchfu waiting"
    (logical
        (bacterial_suspicion (criteria_met yes))
        (bacterial_complication (criteria_met no))
        (watchful_waiting_check (watch_wait ?watch_wait))
        
        (watchful_waiting_check (watch_wait yes)))

    
    ?f1 <-(watchful_waiting (criteria_met unknown))
    => 
    (modify ?f1 (criteria_met yes))
    (println "Reassess in 7 days")
    (assert (start_worsening_no_improvement_check yes)))
"""
env.build(DEFRULE_WATCHFUL_WAITING_CHECK_YES)


# ; RULE: NO eligibility for watchful waiting
# ; *Forward chaining to determine if NOT eligible for watchful waiting.
# ; INPUT:  watchful_waiting_checkv
# ; OUTPUT: (assert (start_severe_infection_hospital_check yes))
DEFRULE_WATCHFUL_WAITING_CHECK_NO = """
(defrule watchful_waiting_check_no "Rule to define if there is NO bacterial complication"
    (logical
        (bacterial_suspicion (criteria_met yes))
        (bacterial_complication (criteria_met no))
        (watchful_waiting_check (watch_wait ?watch_wait))
        
        (watchful_waiting_check (watch_wait no)))

    
    ?f1 <-(watchful_waiting (criteria_met unknown))
    => 
    (modify ?f1 (criteria_met no))
    (assert (start_severe_infection_hospital_check yes)))
"""
env.build(DEFRULE_WATCHFUL_WAITING_CHECK_NO)


### Rule: Severe Infection in Need for Hospitalization Assessment

The defrules in this section support gathering input on whether there is severe infection with need for hospitalization.

If there is severe infection with need for hospitalization: (assert (beta_lactam_allergy_check yes)))
If there is NO severe infection with NO need for hospitalization: (assert (beta_lactam_allergy_check yes)))

In [1765]:
DEFRULE_READ_SEVERE_INFECTION_HOSPITAL_CHECK = """
(defrule reading_input_severe_infection_hospital_check
    (initial_decision_track (criteria_met yes))
    (bacterial_suspicion (criteria_met yes))
    (bacterial_complication (criteria_met no))
    (watchful_waiting (criteria_met no))
    (start_severe_infection_hospital_check yes)
    =>
    (read_assert severe_infection_hospital_check))
"""
env.build(DEFRULE_READ_SEVERE_INFECTION_HOSPITAL_CHECK)


# ; RULE: Assess if severe infection with need for hospitalization
# ; *Forward chaining to determine if severe infection with need for hospitalization
# ; INPUT:  severe_infection_hospital_check
# ; OUTPUT: (assert (beta_lactam_allergy_check yes)))
DEFRULE_SEVERE_INFECTION_HOSPITAL_CHECK_YES = """
(defrule severe_infection_hospital_check_yes "Rule to define if there is severe infection with need for hospitalization"
    (logical
        (initial_decision_track (criteria_met yes))
        (bacterial_suspicion (criteria_met yes))
        (bacterial_complication (criteria_met no))
        (watchful_waiting (criteria_met no))
        (severe_infection_hospital_check (severe_hospital ?severe_hospital))
        
        (severe_infection_hospital_check (severe_hospital yes)))

    
    ?f1 <-(severe_infection_hospital (criteria_met unknown))
    => 
    (modify ?f1 (criteria_met yes))
    (assert (start_beta_lactam_allergy_check yes)))
"""
env.build(DEFRULE_SEVERE_INFECTION_HOSPITAL_CHECK_YES)


# ; RULE: Assess if NO severe infection with NO need for hospitalization
# ; *Forward chaining to determine if NOT severe infection with NO need for hospitalization
# ; INPUT:  severe_infection_hospital_check
# ; OUTPUT: (assert (beta_lactam_allergy_check yes)))
DEFRULE_SEVERE_INFECTION_HOSPITAL_CHECK_NO = """
(defrule severe_infection_hospital_check_no "Rule to define if there is NO severe infection with need for hospitalization"
    (logical
        (initial_decision_track (criteria_met yes))
        (bacterial_suspicion (criteria_met yes))
        (bacterial_complication (criteria_met no))
        (watchful_waiting (criteria_met no))
        (severe_infection_hospital_check (severe_hospital ?severe_hospital))
        
        (severe_infection_hospital_check (severe_hospital no)))

    
    ?f1 <-(severe_infection_hospital (criteria_met unknown))
    => 
    (modify ?f1 (criteria_met no))
    (assert (start_beta_lactam_allergy_check yes)))
"""
env.build(DEFRULE_SEVERE_INFECTION_HOSPITAL_CHECK_NO)

### Rule: Getting Worse or Not Improving in 7 days

The defrules in this section support gathering input on whether, after a week of watchful waiting and symptomatic management, there has been any worsening or no improvement.

If worsening or no improvement: (assert (severe_infection_hospital_check yes))
If there is no worsening and there is improvement: print message: "Keep current management until resolution".


In [1767]:
DEFRULE_READ_WORSENING_NO_IMPROVEMENT_CHECK = """
(defrule reading_input_worsening_no_improvement_check
    (initial_decision_track (criteria_met yes))
    (start_rhinosinusitis yes)
    (rhinosinusitis (criteria_met yes))
    (start_rhinosinusitis_duration yes)
    (bacterial_suspicion (criteria_met yes))
    (bacterial_complication (criteria_met no))
    (start_watch_wait yes)
    (watchful_waiting (criteria_met yes))
    (start_worsening_no_improvement_check yes)
    =>
    (read_assert worsening_no_improvement_check))
"""
env.build(DEFRULE_READ_WORSENING_NO_IMPROVEMENT_CHECK)

# ; RULE: Assess if worsening or no improvement after a week of watchful waiting
# ; *Forward chaining to determine if worsening or no improvement after a week of watchful waiting
# ; INPUT:  worsening_no_improvement_check
# ; OUTPUT: (assert (start_severe_infection_hospital_check yes))
DEFRULE_WORSENING_NO_IMPROVEMENT_CHECK_YES = """
(defrule worsening_no_improvement_check_yes "Rule to define if worsening or no improvement after a week of watchful waiting"
    (logical
        (initial_decision_track (criteria_met yes))
        (rhinosinusitis (criteria_met yes))
        (bacterial_suspicion (criteria_met yes))
        (bacterial_complication (criteria_met no))
        (watchful_waiting (criteria_met yes))
        (worsening_no_improvement_check (worse_noimprove ?worse_noimprove))
        
        (worsening_no_improvement_check (worse_noimprove yes)))

    
    ?f1 <-(worsening_no_improvement (criteria_met unknown))
    => 
    (modify ?f1 (criteria_met yes))
    (assert (start_severe_infection_hospital_check_2 yes)))
"""
env.build(DEFRULE_WORSENING_NO_IMPROVEMENT_CHECK_YES)


# ; RULE: Assess if improvement or no worseningafter a week of watchful waiting
# ; *Forward chaining to determine if no worsening or improvement after a week of watchful waiting
# ; INPUT:  worsening_no_improvement_check
# ; OUTPUT: print message
DEFRULE_WORSENING_NO_IMPROVEMENT_CHECK_NO = """
(defrule worsening_no_improvement_check_no "Rule to define if improvement or no worsening after a week of watchful waiting"
    (logical
        (initial_decision_track (criteria_met yes))
        (rhinosinusitis (criteria_met yes))
        (bacterial_suspicion (criteria_met yes))
        (bacterial_complication (criteria_met no))
        (watchful_waiting (criteria_met yes))
        (worsening_no_improvement_check (worse_noimprove ?worse_noimprove))
        
        (worsening_no_improvement_check (worse_noimprove no)))
    
    ?f1 <-(worsening_no_improvement (criteria_met unknown))
    => 
    (modify ?f1 (criteria_met no))
    (println "Keep current management until resolution"))
"""
env.build(DEFRULE_WORSENING_NO_IMPROVEMENT_CHECK_NO)


### Rule: Checking for Beta-Lactam Antibiotics Allergy in non-hospitalized patients

The defrules in this section support gathering input on whether outpatient patient has beta-lactam allergy.
If outpatient beta-lactam allergy yes: print message and (assert (start_clinical_improvement_check yes))
If outpatient beta-lactam allergy no: print message and (assert (start_clinical_improvement_check yes))


In [1769]:
DEFRULE_READ_BETA_LACTAM_ALLERGY_OUTPATIENT = """
(defrule reading_input_beta_lactam_allergy_outpatient
    (initial_decision_track (criteria_met yes))
    (rhinosinusitis (criteria_met yes))
    (bacterial_suspicion (criteria_met yes))
    (bacterial_complication (criteria_met no))
    (watchful_waiting (criteria_met no))
    (start_severe_infection_hospital_check yes)
    (severe_infection_hospital (criteria_met no))
    (start_beta_lactam_allergy_check yes)
    =>
    (read_assert beta_lactam_allergy_check))
"""
env.build(DEFRULE_READ_BETA_LACTAM_ALLERGY_OUTPATIENT)


# ; RULE: Assess if outpatient case has beta-lactam allergy
# ; *Forward chaining to determine if outpatient case has beta-lactam allergy
# ; INPUT:  beta_lactam_allergy_check
# ; OUTPUT: print message and (assert (start_clinical_improvement_check yes)))
DEFRULE_BETA_LACTAM_ALLERGY_OUTPATIENT_YES = """
(defrule beta_lactam_allergy_outpatient_yes "Rule to define if outpatient case has beta-lactam allergy"
    (logical
        (initial_decision_track (criteria_met yes))
        (rhinosinusitis (criteria_met yes))
        (bacterial_suspicion (criteria_met yes))
        (bacterial_complication (criteria_met no))
        (watchful_waiting (criteria_met no))
        (severe_infection_hospital (criteria_met no))
        
        (beta_lactam_allergy_check (allergy ?allergy))
        (beta_lactam_allergy_check (allergy yes)))

    
    ?f1 <-(beta_lactam_allergy (criteria_met unknown))
    => 
    (modify ?f1 (criteria_met yes))
    (println "Consider oral respiratory quinolone or doxicycline")
    (assert (start_clinical_improvement_check yes)))
"""
env.build(DEFRULE_BETA_LACTAM_ALLERGY_OUTPATIENT_YES)


# ; RULE: Assess if outpatient case has NOT beta-lactam allergy
# ; *Forward chaining to determine if outpatient case has NOT beta-lactam allergy
# ; INPUT:  beta_lactam_allergy_check
# ; OUTPUT: print message and (assert (start_clinical_improvement_check yes)))
DEFRULE_BETA_LACTAM_ALLERGY_OUTPATIENT_NO = """
(defrule beta_lactam_allergy_outpatient_no "Rule to define if outpatient case has NOT beta-lactam allergy"
    (logical
        (initial_decision_track (criteria_met yes))
        (rhinosinusitis (criteria_met yes))
        (bacterial_suspicion (criteria_met yes))
        (bacterial_complication (criteria_met no))
        (watchful_waiting (criteria_met no))
        (severe_infection_hospital (criteria_met no))
        
        (beta_lactam_allergy_check (allergy ?allergy))
        (beta_lactam_allergy_check (allergy no)))

    
    ?f1 <-(beta_lactam_allergy (criteria_met unknown))
    => 
    (modify ?f1 (criteria_met no))
    (println "Consider oral amoxacilin + clavulanate 875mg + 125mg 12/12h 5-7 days")
    (assert (start_clinical_improvement_check yes)))
"""
env.build(DEFRULE_BETA_LACTAM_ALLERGY_OUTPATIENT_NO)

### Rule: Checking for Beta-Lactam Antibiotics Allergy in Hospitalized patients

The defrules in this section support gathering input on whether inpatient patient has beta-lactam allergy.
If inpatient beta-lactam allergy yes: print message and (assert (start_clinical_improvement_check yes))
If inpatient beta-lactam allergy no: print message and (assert (start_clinical_improvement_check yes))

In [1771]:
DEFRULE_READ_BETA_LACTAM_ALLERGY_INPATIENT = """
(defrule reading_input_beta_lactam_allergy_inpatient
    (initial_decision_track (criteria_met yes))
    (rhinosinusitis (criteria_met yes))
    (bacterial_suspicion (criteria_met yes))
    (bacterial_complication (criteria_met no))
    (watchful_waiting (criteria_met no))
    (severe_infection_hospital (criteria_met yes))
    (start_beta_lactam_allergy_check yes)
    =>
    (read_assert beta_lactam_allergy_check))
"""
env.build(DEFRULE_READ_BETA_LACTAM_ALLERGY_INPATIENT)


# ; RULE: Assess if inpatient case has beta-lactam allergy
# ; *Forward chaining to determine if inpatient case has beta-lactam allergy
# ; INPUT:  beta_lactam_allergy_check
# ; OUTPUT: print message and (assert (start_clinical_improvement_check yes)))
DEFRULE_BETA_LACTAM_ALLERGY_INPATIENT_YES = """
(defrule beta_lactam_allergy_inpatient_yes "Rule to define if inpatient case has beta-lactam allergy"
    (logical
        (initial_decision_track (criteria_met yes))
        (rhinosinusitis (criteria_met yes))
        (bacterial_suspicion (criteria_met yes))
        (bacterial_complication (criteria_met no))
        (watchful_waiting (criteria_met no))
        (severe_infection_hospital (criteria_met yes))
        
        (beta_lactam_allergy_check (allergy ?allergy))
        (beta_lactam_allergy_check (allergy yes)))

    
    ?f1 <-(beta_lactam_allergy (criteria_met unknown))
    => 
    (modify ?f1 (criteria_met yes))
    (println "Consider Levofloxacin 750mg IV 24/24h 5-7days; check for bacterial resistance")
    (assert (start_clinical_improvement_check yes)))
"""
env.build(DEFRULE_BETA_LACTAM_ALLERGY_INPATIENT_YES)


# ; RULE: Assess if inpatient case has NOT beta-lactam allergy
# ; *Forward chaining to determine if inpatient case has NOT beta-lactam allergy
# ; INPUT:  beta_lactam_allergy_check
# ; OUTPUT: print message and (assert (start_clinical_improvement_check yes)))
DEFRULE_BETA_LACTAM_ALLERGY_INPATIENT_NO = """
(defrule beta_lactam_allergy_inpatient_no "Rule to define if inpatient case has NOT beta-lactam allergy"
    (logical
        (initial_decision_track (criteria_met yes))
        (rhinosinusitis (criteria_met yes))
        (bacterial_suspicion (criteria_met yes))
        (bacterial_complication (criteria_met no))
        (watchful_waiting (criteria_met no))
        (severe_infection_hospital (criteria_met yes))
        
        (beta_lactam_allergy_check (allergy ?allergy))
        (beta_lactam_allergy_check (allergy no)))

    
    ?f1 <-(beta_lactam_allergy (criteria_met unknown))
    => 
    (modify ?f1 (criteria_met no))
    (println "In hospital IV beta-lactam;check for bacterial resistance ")
    (assert (start_clinical_improvement_check yes)))
"""
env.build(DEFRULE_BETA_LACTAM_ALLERGY_INPATIENT_NO)

### Rule: Checking for Clinical Improvement after first treatment

The defrules in this section support gathering input on whether the patient got better after first management.
If clinical_improvement yes: print message "Keep symptomatics(analgesics, antipiretics) and nasal irrigation with 0.9% saline until symptom resolution")
If clinical_improvement no: print message "Order imaging exam to assess for complications;Consider changing antibiotic therapy, if possible, based on resistance profile;Refer for ENT evaluation. Re-consider differential diagnosis, consider fungal causes. If recurrent, assess for cystic fibrosis, immotile cilia disorders, ciliary dyskinesia, immune deficiency, prior history of sinus surgery, and anatomic abnormalities (eg, deviated nasal septum)."


In [1773]:
DEFRULE_READ_CLINICAL_IMPROVEMENT_CHECK = """
(defrule reading_input_clinical_improvement_check
    (initial_decision_track (criteria_met yes))
    (rhinosinusitis (criteria_met yes))
    (bacterial_suspicion (criteria_met yes))
    (bacterial_complication (criteria_met no))
    (watchful_waiting (criteria_met no))
    (start_severe_infection_hospital_check yes)
    (start_beta_lactam_allergy_check yes)
    (start_clinical_improvement_check yes)
    =>
    (read_assert clinical_improvement_check))
"""
env.build(DEFRULE_READ_CLINICAL_IMPROVEMENT_CHECK)


# ; RULE: Assess if there was clinical improvement after first management
# ; *Forward chaining to determine if there was clinical improvement after first management
# ; INPUT:  clinical_improvement_check
# ; OUTPUT: print message
DEFRULE_CLINICAL_IMPROVEMENT_CHECK_YES = """
(defrule clinical_improvement_check_yes "Rule to define if there was clinical improvement after first management"
    (logical
        (initial_decision_track (criteria_met yes))
        (rhinosinusitis (criteria_met yes))
        (bacterial_suspicion (criteria_met yes))
        (bacterial_complication (criteria_met no))
        (watchful_waiting (criteria_met no))
        (start_severe_infection_hospital_check yes)
        (start_beta_lactam_allergy_check yes)
        (start_clinical_improvement_check yes)
        
        (clinical_improvement_check(better ?better))
        (clinical_improvement_check (better yes)))

    
    ?f1 <-(clinical_improvement (criteria_met unknown))
    => 
    (modify ?f1 (criteria_met yes))
    (println "Keep symptomatics (analgesics, antipiretics) and nasal irrigation with 0.9% saline until symptom resolution"))
"""
env.build(DEFRULE_CLINICAL_IMPROVEMENT_CHECK_YES)


# ; RULE: Assess if there was NO clinical improvement after first management
# ; *Forward chaining to determine if there was NO clinical improvement after first management
# ; INPUT:  clinical_improvement_check
# ; OUTPUT: print message
DEFRULE_CLINICAL_IMPROVEMENT_CHECK_NO = """
(defrule clinical_improvement_check_no "Rule to define if there was NO clinical improvement after first management"
    (logical
        (initial_decision_track (criteria_met yes))
        (rhinosinusitis (criteria_met yes))
        (bacterial_suspicion (criteria_met yes))
        (bacterial_complication (criteria_met no))
        (watchful_waiting (criteria_met no))
        (start_severe_infection_hospital_check yes)
        (start_beta_lactam_allergy_check yes)
        (start_clinical_improvement_check yes)
        
        (clinical_improvement_check (better ?better))
        (clinical_improvement_check (better no)))

    
    ?f1 <-(clinical_improvement (criteria_met unknown))
    => 
    (modify ?f1 (criteria_met no))
    (println "Order imaging exam to assess for complications; Consider changing antibiotic therapy, if possible, based on resistance profile; Refer for ENT evaluation. Re-consider differential diagnosis, consider fungal causes. If recurrent, assess for cystic fibrosis, immotile cilia disorders, ciliary dyskinesia, immune deficiency, prior history of sinus surgery, and anatomic abnormalities (eg, deviated nasal septum)."))
"""
env.build(DEFRULE_CLINICAL_IMPROVEMENT_CHECK_NO)

### Rule 2: Severe Infection in Need for Hospitalization Assessment post watchful waiting

The defrules in this section support gathering input on whether there is severe infection with need for hospitalization.
This rule will trigger if the patient initially did a week of watchful waiting and did not improve or got worse after it.

If there is severe infection with need for hospitalization: (assert (beta_lactam_allergy_check yes)))
If there is NO severe infection with NO need for hospitalization: (assert (beta_lactam_allergy_check yes)))

In [1775]:
DEFRULE_READ_SEVERE_INFECTION_HOSPITAL_EVAL_2 = """
(defrule reading_input_severe_infection_hospital_eval_2
    (worsening_no_improvement (criteria_met yes))
    (start_severe_infection_hospital_check_2 yes)
    =>
    (read_assert severe_infection_hospital_check))
"""
env.build(DEFRULE_READ_SEVERE_INFECTION_HOSPITAL_EVAL_2)


# ; RULE: Assess if severe infection with need for hospitalization
# ; *Forward chaining to determine if severe infection with need for hospitalization
# ; INPUT:  severe_infection_hospital_check
# ; OUTPUT: (assert (beta_lactam_allergy_check yes)))
DEFRULE_SEVERE_INFECTION_HOSPITAL_EVAL_YES_2 = """
(defrule severe_infection_hospital_eval_yes_2 "Rule to define if there is severe infection with need for hospitalization"
    (logical
        (worsening_no_improvement (criteria_met yes))
        (severe_infection_hospital_check (severe_hospital ?severe_hospital))
        
        (severe_infection_hospital_check (severe_hospital yes)))

    
    ?f1 <-(severe_infection_hospital (criteria_met unknown))
    => 
    (modify ?f1 (criteria_met yes))
    (assert (start_beta_lactam_allergy_check_2 yes)))
"""
env.build(DEFRULE_SEVERE_INFECTION_HOSPITAL_EVAL_YES_2)


# ; RULE: Assess if NO severe infection with NO need for hospitalization
# ; *Forward chaining to determine if NOT severe infection with NO need for hospitalization
# ; INPUT:  severe_infection_hospital_check
# ; OUTPUT: (assert (beta_lactam_allergy_check yes)))
DEFRULE_SEVERE_INFECTION_HOSPITAL_EVAL_NO_2 = """
(defrule severe_infection_hospital_eval_no_2 "Rule to define if there is NO severe infection with need for hospitalization"
    (logical
        (worsening_no_improvement (criteria_met yes))
        (severe_infection_hospital_check (severe_hospital ?severe_hospital))
        
        (severe_infection_hospital_check (severe_hospital no)))

    
    ?f1 <-(severe_infection_hospital (criteria_met unknown))
    => 
    (modify ?f1 (criteria_met no))
    (assert (start_beta_lactam_allergy_check_2 yes)))
"""
env.build(DEFRULE_SEVERE_INFECTION_HOSPITAL_EVAL_NO_2)

### Rule 2: Checking for Beta-Lactam Antibiotics Allergy in non-hospitalized patients after watchful waiting

The defrules in this section support gathering input on whether outpatient patient has beta-lactam allergy, after watchful waiting that did not improve or became worse
If outpatient beta-lactam allergy yes: print message and (assert (start_clinical_improvement_check yes))
If outpatient beta-lactam allergy no: print message and (assert (start_clinical_improvement_check yes))


In [1777]:
DEFRULE_READ_BETA_LACTAM_ALLERGY_OUTPATIENT_2= """
(defrule reading_input_beta_lactam_allergy_outpatient_2
    (watchful_waiting (criteria_met yes))
    (worsening_no_improvement (criteria_met yes))
    (start_severe_infection_hospital_check_2 yes)
    (severe_infection_hospital (criteria_met no))
    (start_beta_lactam_allergy_check_2 yes)
    =>
    (read_assert beta_lactam_allergy_check))
"""
env.build(DEFRULE_READ_BETA_LACTAM_ALLERGY_OUTPATIENT_2)


# ; RULE: Assess if outpatient case has beta-lactam allergy
# ; *Forward chaining to determine if outpatient case has beta-lactam allergy
# ; INPUT:  beta_lactam_allergy_check
# ; OUTPUT: print message and (assert (start_clinical_improvement_check yes)))
DEFRULE_BETA_LACTAM_ALLERGY_OUTPATIENT_YES_2 = """
(defrule beta_lactam_allergy_outpatient_yes_2 "Rule to define if outpatient case has beta-lactam allergy"
    (logical
        (watchful_waiting (criteria_met yes))
        (worsening_no_improvement (criteria_met yes))
        (severe_infection_hospital (criteria_met no))
        
        (beta_lactam_allergy_check (allergy ?allergy))
        (beta_lactam_allergy_check (allergy yes)))

    
    ?f1 <-(beta_lactam_allergy (criteria_met unknown))
    => 
    (modify ?f1 (criteria_met yes))
    (println "Consider oral respiratory quinolone or doxicycline")
    (assert (start_clinical_improvement_check yes)))
"""
env.build(DEFRULE_BETA_LACTAM_ALLERGY_OUTPATIENT_YES_2)


# ; RULE: Assess if outpatient case has NOT beta-lactam allergy
# ; *Forward chaining to determine if outpatient case has NOT beta-lactam allergy
# ; INPUT:  beta_lactam_allergy_check
# ; OUTPUT: print message and (assert (start_clinical_improvement_check yes)))
DEFRULE_BETA_LACTAM_ALLERGY_OUTPATIENT_NO_2 = """
(defrule beta_lactam_allergy_outpatient_no_2 "Rule to define if outpatient case has NOT beta-lactam allergy"
    (logical
        (watchful_waiting (criteria_met yes))
        (worsening_no_improvement (criteria_met yes))
        (severe_infection_hospital (criteria_met no))
        
        (beta_lactam_allergy_check (allergy ?allergy))
        (beta_lactam_allergy_check (allergy no)))

    
    ?f1 <-(beta_lactam_allergy (criteria_met unknown))
    => 
    (modify ?f1 (criteria_met no))
    (println "Consider oral amoxacilin + clavulanate 875mg + 125mg 12/12h 5-7 days")
    (assert (start_clinical_improvement_check yes)))
"""
env.build(DEFRULE_BETA_LACTAM_ALLERGY_OUTPATIENT_NO_2)

### Rule 2: Checking for Beta-Lactam Antibiotics Allergy in Hospitalized patients after watchful waiting

The defrules in this section support gathering input on whether inpatient patient has beta-lactam allergy, after watchful waiting that did not improve or became worse

If inpatient beta-lactam allergy yes: print message and (assert (start_clinical_improvement_check yes))
If inpatient beta-lactam allergy no: print message and (assert (start_clinical_improvement_check yes))

In [1779]:
DEFRULE_READ_BETA_LACTAM_ALLERGY_INPATIENT_2 = """
(defrule reading_input_beta_lactam_allergy_inpatient_2
    (watchful_waiting (criteria_met yes))
    (worsening_no_improvement (criteria_met yes))
    (start_severe_infection_hospital_check_2 yes)
    (severe_infection_hospital (criteria_met yes))
    (start_beta_lactam_allergy_check_2 yes)
    =>
    (read_assert beta_lactam_allergy_check))
"""
env.build(DEFRULE_READ_BETA_LACTAM_ALLERGY_INPATIENT_2)


# ; RULE: Assess if inpatient case has beta-lactam allergy
# ; *Forward chaining to determine if inpatient case has beta-lactam allergy
# ; INPUT:  beta_lactam_allergy_check
# ; OUTPUT: print message and (assert (start_clinical_improvement_check yes)))
DEFRULE_BETA_LACTAM_ALLERGY_INPATIENT_YES_2 = """
(defrule beta_lactam_allergy_inpatient_yes_2 "Rule to define if inpatient case has beta-lactam allergy"
    (logical
        (watchful_waiting (criteria_met yes))
        (worsening_no_improvement (criteria_met yes))
        (severe_infection_hospital (criteria_met yes))
        
        (beta_lactam_allergy_check (allergy ?allergy))
        (beta_lactam_allergy_check (allergy yes)))

    
    ?f1 <-(beta_lactam_allergy (criteria_met unknown))
    => 
    (modify ?f1 (criteria_met yes))
    (println "Consider Levofloxacin 750mg IV 24/24h 5-7days; check for bacterial resistance")
    (assert (start_clinical_improvement_check yes)))
"""
env.build(DEFRULE_BETA_LACTAM_ALLERGY_INPATIENT_YES_2)


# ; RULE: Assess if inpatient case has NOT beta-lactam allergy
# ; *Forward chaining to determine if inpatient case has NOT beta-lactam allergy
# ; INPUT:  beta_lactam_allergy_check
# ; OUTPUT: print message and (assert (start_clinical_improvement_check yes)))
DEFRULE_BETA_LACTAM_ALLERGY_INPATIENT_NO_2 = """
(defrule beta_lactam_allergy_inpatient_no_2 "Rule to define if inpatient case has NOT beta-lactam allergy"
    (logical
        (watchful_waiting (criteria_met yes))
        (worsening_no_improvement (criteria_met yes))
        (severe_infection_hospital (criteria_met yes))
        
        (beta_lactam_allergy_check (allergy ?allergy))
        (beta_lactam_allergy_check (allergy no)))

    
    ?f1 <-(beta_lactam_allergy (criteria_met unknown))
    => 
    (modify ?f1 (criteria_met no))
    (println "In hospital IV beta-lactam;check for bacterial resistance ")
    (assert (start_clinical_improvement_check_2 yes)))
"""
env.build(DEFRULE_BETA_LACTAM_ALLERGY_INPATIENT_NO_2)

### Rule 2: Checking for Clinical Improvement after first treatment after watchful waiting

The defrules in this section support gathering input on whether the patient got better after first management, after watchful waiting that did not improve or became worse

If clinical_improvement yes: print message "Keep symptomatics(analgesics, antipiretics) and nasal irrigation with 0.9% saline until symptom resolution")
If clinical_improvement no: print message "Order imaging exam to assess for complications;Consider changing antibiotic therapy, if possible, based on resistance profile;Refer for ENT evaluation. Re-consider differential diagnosis, consider fungal causes. If recurrent, assess for cystic fibrosis, immotile cilia disorders, ciliary dyskinesia, immune deficiency, prior history of sinus surgery, and anatomic abnormalities (eg, deviated nasal septum)."

In [1781]:
DEFRULE_READ_CLINICAL_IMPROVEMENT_CHECK_2 = """
(defrule reading_input_clinical_improvement_check_2
    (start_clinical_improvement_check_2 yes)
    =>
    (read_assert clinical_improvement_check))
"""
env.build(DEFRULE_READ_CLINICAL_IMPROVEMENT_CHECK_2)


# ; RULE: Assess if there was clinical improvement after first management
# ; *Forward chaining to determine if there was clinical improvement after first management
# ; INPUT:  clinical_improvement_check
# ; OUTPUT: print message
DEFRULE_CLINICAL_IMPROVEMENT_CHECK_YES_2 = """
(defrule clinical_improvement_check_yes_2 "Rule to define if there was clinical improvement after first management"
    (logical
        (initial_decision_track (criteria_met yes))
        (rhinosinusitis (criteria_met yes))
        (bacterial_suspicion (criteria_met yes))
        (bacterial_complication (criteria_met no))
        (watchful_waiting (criteria_met yes))
        (worsening_no_improvement (criteria_met yes))
        (start_severe_infection_hospital_check yes)
        (start_beta_lactam_allergy_check yes)
        (start_clinical_improvement_check_2 yes)
        
        (clinical_improvement_check(better ?better))
        (clinical_improvement_check (better yes)))

    
    ?f1 <-(clinical_improvement (criteria_met unknown))
    => 
    (modify ?f1 (criteria_met yes))
    (println "Keep symptomatics (analgesics, antipiretics) and nasal irrigation with 0.9% saline until symptom resolution"))
"""
env.build(DEFRULE_CLINICAL_IMPROVEMENT_CHECK_YES_2)


# ; RULE: Assess if there was NO clinical improvement after first management
# ; *Forward chaining to determine if there was NO clinical improvement after first management
# ; INPUT:  clinical_improvement_check
# ; OUTPUT: print message
DEFRULE_CLINICAL_IMPROVEMENT_CHECK_NO_2 = """
(defrule clinical_improvement_check_no_2 "Rule to define if there was NO clinical improvement after first management"
    (logical
        (initial_decision_track (criteria_met yes))
        (rhinosinusitis (criteria_met yes))
        (acute_rhinosinusitis (criteria_met yes))
        (bacterial_suspicion (criteria_met yes))
        (bacterial_complication (criteria_met no))
        (watchful_waiting (criteria_met yes))
        (worsening_no_improvement (criteria_met yes))
        (start_severe_infection_hospital_check yes)
        (start_beta_lactam_allergy_check yes)
        (start_clinical_improvement_check yes)
        
        (clinical_improvement_check (better ?better))
        (clinical_improvement_check (better no)))

    
    ?f1 <-(clinical_improvement (criteria_met unknown))
    => 
    (modify ?f1 (criteria_met no))
    (println "Order imaging exam to assess for complications; Consider changing antibiotic therapy, if possible, based on resistance profile; Refer for ENT evaluation. Re-consider differential diagnosis, consider fungal causes. If recurrent, assess for cystic fibrosis, immotile cilia disorders, ciliary dyskinesia, immune deficiency, prior history of sinus surgery, and anatomic abnormalities (eg, deviated nasal septum)."))
"""
env.build(DEFRULE_CLINICAL_IMPROVEMENT_CHECK_NO_2)

In [1782]:
env.reset();
env.run();

What is the patient's age (in years)?:  68
What is the patient's sex (male or female)?:  male
Does the patient have predominance of upper airway symptoms (yes or no)?:  yes
Based on clinical assessment, is there suspicion for rhinossinusitis (yes or no)?:  yes


___________
    This decision track is adequate for this case.
    Apart from using this decision track, also consider differencial diagnosis, ruling them out, such as:
    dental pain (if unilateral facial pain), neuralgic atypical facial pain or trigeminal neuralgia, 
    temporomandibular joint pain, migrain, rhinitis (allergic, eosinophilic nonallergic), 
    vasomotor headaches, neoplastic conditions. 
___________


Is there purulent nasal discharge (yes no)?:  yes
Is there purulent anterior nasal discharge (yes no)?:  yes
Is there purulent or clear posterior nasal discharge (yes no)?:  yes
Is there nasal congestion or obstruction (yes no)?:  yes
Is there facial congestion or fullness (yes no)?:  yes
Is there facial pain or pressure (yes no)?:  no
Is there hyposmia or anosmia (yes no)?:  no
Is there fever (yes no)?:  no
Is there headache (yes no)?:  no
Is there ear pain, pressure and fullness (yes no)?:  yes
Is there halitosis (yes no)?:  no
Is there dental pain (yes no)?:  no
Is there cough (yes no)?:  no
Is there fatigue (yes no)?:  yes


This patient matches the criteria for rhinosinusitis.


Is the duration of this presentation four weeks or less (yes no)?: yes
How many days has it been since the beggining of this presentation(in days)?: 18
Has there been any improvement (yes no)?: yes
Has it been progresivelly worsening (yes no)?: no
Was the patient improving at first but than started worsening again (yes no)?: yes
Have the symptoms been severe for at least 3 (three) days (yes no): no
Based on your clinical assessment,is there suspicion for bacterial infection(yes no)?: yes


...

Prescribe symptomatics (analgesics, antipiretics) and nasal irrigation with 0.9% saline. Consider intranasal corticosteroids for symptomatic relief


Based on your clinical assessment, is this patient eligible for watchful waiting (yes no)?:  no
Is this a severe case in need of hospitalization (yes no)?:  yes
Is the patient allergic to beta-lactam antibiotics (yes no)?:  yes


Consider Levofloxacin 750mg IV 24/24h 5-7days; check for bacterial resistance


Is there clinical improvement after initial treatment(yes no)?:  no


Order imaging exam to assess for complications; Consider changing antibiotic therapy, if possible, based on resistance profile; Refer for ENT evaluation. Re-consider differential diagnosis, consider fungal causes. If recurrent, assess for cystic fibrosis, immotile cilia disorders, ciliary dyskinesia, immune deficiency, prior history of sinus surgery, and anatomic abnormalities (eg, deviated nasal septum).


In [1783]:
print_facts(env)

(initial_decision_track (criteria_met yes))
(rhinosinusitis (criteria_met yes))
(acute_rhinosinusitis (criteria_met yes))
(bacterial_suspicion (criteria_met yes))
(chronic_rhinosinusitis (criteria_met unknown))
(sa_acute_ttm_need (criteria_met unknown))
(bacterial_complication (criteria_met no))
(watchful_waiting (criteria_met no))
(worsening_no_improvement (criteria_met unknown))
(severe_infection_hospital (criteria_met yes))
(beta_lactam_allergy (criteria_met yes))
(clinical_improvement (criteria_met no))
(patient (age_is 68) (sex_is male))
(upper_airway_predom (with_upper_airway_symp_predom yes))
(rhinosinusitis_suspicion (with_rhinosinusitis_suspicion yes))
(start_rhinosinusitis yes)
(symptoms (purulent_nasal_discharge yes))
(major_symptoms (anterior_nasal_discharge yes) (posterior_nasal_discharge yes) (nasal_congestion_obstruction yes) (facial_congestion_fullness yes) (facial_pain_pressure no) (hyposmia_anosmia no) (fever no))
(minor_symptoms (headache no) (ear_pain_pressure_fulln